In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 22 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

20 March 2023 ; 23:11:38


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,1,1
1,Bobby Portis,PF,27,MIL,60,19,26.0,5.7,11.3,0.501,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1
2,Brook Lopez,C,34,MIL,69,69,30.7,6.0,11.5,0.521,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0
3,Giannis Antetokounmpo,PF,28,MIL,56,56,32.5,11.1,20.4,0.546,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0
4,Grayson Allen,SG,27,MIL,65,63,27.8,3.5,7.9,0.441,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,31.6,10.506250,18.065625,24.7,1,0.875060
1,Giannis Antetokounmpo,MIL,28.7,6.882143,11.275000,31.3,1,0.694040
2,Joel Embiid,PHI,31.6,8.056140,15.680702,33.5,2,0.658790
3,Luka Dončić,DAL,29.5,8.631579,13.378947,33.0,6,0.477746
4,Damian Lillard,POR,26.9,6.905263,13.522807,32.2,13,0.293348


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)